<a href="https://www.kaggle.com/code/hakudan/flood-prediction?scriptVersionId=177798968" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import time
import xgboost as xgb
import lightgbm as lgb
import catboost
import optuna
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, SplineTransformer
from sklearn.linear_model import Ridge, LinearRegression, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.metrics import r2_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


## 1. Loading data

In [2]:
df_train = pd.read_csv('../input/playground-series-s4e5/train.csv', index_col='id')
df_test = pd.read_csv('../input/playground-series-s4e5/test.csv', index_col='id')

# remove id from dataset
X_train = df_train.drop('FloodProbability', axis=1)
# X_train_ini = df_train.drop(['id'], axis=1)
y_train = df_train.FloodProbability

initial_features = list(X_train.columns)
# test_set = df_test.drop('id', axis=1)

# Split training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=22)

In [ ]:
print('Number of Training Examples = {}'.format(df_train.shape[0]))
print('Number of Test Examples = {}\n'.format(df_test.shape[0]))
print('Training X Shape = {}'.format(df_train.shape))
print('Test X Shape = {}\n'.format(df_test.shape))
print(df_train.columns)
print(df_test.columns)

## 2. Exploratory Data Analysis

### 1.1. Overview
All features are integer.
There's no missing values in both train and test set.

In [ ]:
X_train.head()

In [ ]:
print('Shape of training dataset is: {}'.format(X_train.shape))
print('Shape of test dataset is: {}'.format(df_test.shape))

In [ ]:
X_train.info()

In [ ]:
df_test.info()

In [ ]:
X_train.describe().style.format("{:.5f}")

In [ ]:
df_test.describe().style.format("{:.5f}")

### 1.3. Duplicate values

In [ ]:
train_dup = df_train.duplicated().sum()
test_dup = df_test.duplicated().sum()
print('There is {} duplicate value(s) in training set'.format(train_dup))
print('There is {} duplicate value(s) in training set'.format(test_dup))

### 1.4. Distribution
All features look almost the same and have right-skewed distribution in both training and test set.

The target has a normal distribution with mean at 0.5.

In [ ]:
# plot each feature of dataset
def plot_feature(data, column, ax, color):  
    sns.countplot(data, x=column, color=color, ax=ax) 
    ax.set_ylabel('Count') 

fig, axs = plt.subplots(5, 4, figsize=(15, 12)) 
for ax, column in zip(axs.flat, X_train.columns):
    plot_feature(X_train, column, ax, 'royalblue')
plt.tight_layout() 
plt.show()

fig, axs = plt.subplots(5, 4, figsize=(15, 12)) 
for ax, column in zip(axs.flat, df_test.columns):
    plot_feature(df_test, column, ax, 'orange')
plt.tight_layout() 
plt.show()

In [ ]:
sns.displot(y_train, x=y_train, color='royalblue') 
plt.show()

### 1.5. Correlation
The correlation matrix shows that there is no correltion between features, but all features are correlated with the target.

In [ ]:
# calculate correlation matrix
corr = df_train.corr(method='pearson')
# plot the matrix
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corr,cmap="YlGnBu", annot=True, fmt=".2f", )
plt.title('Correlation Matrix')
plt.show

## 2. Fitting models

In [3]:
# sorting
for df in [X_train, X_val, df_test]:
    df = np.sort(df.values, axis=1)
sorted_features = [f"sort_{i}" for i in np.arange(len(initial_features))]
# add new features
for df in [X_train, X_val, df_test]:
    df[sorted_features] = np.sort(df[initial_features], axis=1)
    df['fsum'] = df.sum(axis=1)
    df['special1'] = df['fsum'].isin(np.arange(72, 76))
    df = df.drop(initial_features, axis=1, inplace=True)
    
def evaluate_model(model):
    """Trains a model, calculates R² score, and optionally prints training time.

    Args:
        model: The model object to fit (e.g., lgb.LGBMRegressor())
        X_train: Training features
        y_train: Training target values
        X_test: Test features
        y_test: Test target values
    """
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()

    r2_score = model.score(X_val, y_val)

    elapsed_time = end_time - start_time
    print('R2 on test set: {} |  Training Time: {:.2f} seconds'.format(r2_score, elapsed_time))

## 2.1. Linear models

In [4]:
linear_model = make_pipeline(StandardScaler(), LinearRegression())
evaluate_model(linear_model)

R2 on test set: 0.8446255680542413 |  Training Time: 2.59 seconds


In [8]:
ridge_model = make_pipeline(StandardScaler(), Ridge())
evaluate_model(ridge_model)

lasso_model = make_pipeline(StandardScaler(), Lasso())
evaluate_model(lasso_model)

elasnet_model = make_pipeline(StandardScaler(), ElasticNet())
evaluate_model(elasnet_model) 

R2 on test set: 0.8444426052458149 |  Training Time: 0.51 seconds
R2 on test set: -2.68341710629727e-07 |  Training Time: 0.54 seconds
R2 on test set: -2.68341710629727e-07 |  Training Time: 0.55 seconds


#### It seems that the Ridge Regression model is performing significantly better than the Lasso Regression model and Elastic Net

In [ ]:
poly_ridge_model = make_pipeline(StandardScaler(),
                      PolynomialFeatures(degree=2),
                      Ridge())
evaluate_model(poly_ridge_model) 

spline_ridge_model = make_pipeline(StandardScaler(),
                      SplineTransformer(),
                      Ridge())
evaluate_model(spline_ridge_model) 

## 2.2. Tree-based models

In [32]:
def dtree_objective(trial):
    params = {
        'criterion': trial.suggest_categorical('criterion', ['friedman_mse']),
        'max_depth': trial.suggest_int('max_depth', 12, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 80),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 60, 90),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 100, 1000),
        
#             'n_estimators': trial.suggest_int('n_estimators', 300, 1200),
#             'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
#             'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
#             'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
#             'subsample': trial.suggest_float('subsample', 0.25, 1.0),
            }
    model =  DecisionTreeRegressor(**params, random_state=22)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    r2 = model.score(X_val, y_val)
    return r2
    
dtree_study = optuna.create_study(direction='maximize')
dtree_study.optimize(dtree_objective, n_trials=100)

[I 2024-05-15 04:45:53,703] A new study created in memory with name: no-name-6e77523d-978e-49b1-a747-bf34a4505887
[I 2024-05-15 04:45:57,861] Trial 0 finished with value: 0.8683625930542718 and parameters: {'criterion': 'friedman_mse', 'max_depth': 22, 'min_samples_split': 67, 'min_samples_leaf': 64, 'max_leaf_nodes': 381}. Best is trial 0 with value: 0.8683625930542718.
[I 2024-05-15 04:46:01,598] Trial 1 finished with value: 0.8682234934196512 and parameters: {'criterion': 'friedman_mse', 'max_depth': 22, 'min_samples_split': 55, 'min_samples_leaf': 71, 'max_leaf_nodes': 207}. Best is trial 0 with value: 0.8683625930542718.
[I 2024-05-15 04:46:06,320] Trial 2 finished with value: 0.8681296171953206 and parameters: {'criterion': 'friedman_mse', 'max_depth': 34, 'min_samples_split': 21, 'min_samples_leaf': 84, 'max_leaf_nodes': 986}. Best is trial 0 with value: 0.8683625930542718.
[I 2024-05-15 04:46:10,138] Trial 3 finished with value: 0.8682525573913966 and parameters: {'criterion': 

In [33]:
optuna.visualization.plot_parallel_coordinate(dtree_study)

In [37]:
# 0.8684127407376179
# best_decisionTree = dtree_study.best_params
best_decisionTree = {'criterion': 'friedman_mse',
 'max_depth': 41,
 'min_samples_split': 24,
 'min_samples_leaf': 76,
 'max_leaf_nodes': 435}

{'criterion': 'friedman_mse',
 'max_depth': 41,
 'min_samples_split': 24,
 'min_samples_leaf': 76,
 'max_leaf_nodes': 435}

In [36]:
# Decision Tree
# params are got after tunning
dt_model = DecisionTreeRegressor(**best_decisionTree,random_state=22) 
evaluate_model(dt_model) 

R2 on test set: 0.8684127407376179 |  Training Time: 4.35 seconds


In [45]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 140, 300),
        'criterion': trial.suggest_categorical('criterion', ['friedman_mse', 'squared_error', 'poisson']),
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', None]),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 1e-8, 0.1, log=True),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 25, 100),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 1e-9, 10.0, log=True)
            }
    model =  RandomForestRegressor(**params, random_state=22)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    r2 = model.score(X_val, y_val)
    return r2
    
dtree_study = optuna.create_study(direction='maximize')
dtree_study.optimize(objective, n_trials=20)

[I 2024-05-15 10:03:58,726] A new study created in memory with name: no-name-76119165-76f4-49ed-88c4-85989e79453e
[I 2024-05-15 10:04:21,981] Trial 0 finished with value: -2.2487829376416357e-07 and parameters: {'n_estimators': 184, 'criterion': 'poisson', 'max_depth': 97, 'min_samples_leaf': 71, 'max_features': 'sqrt', 'min_weight_fraction_leaf': 2.651323902664541e-06, 'max_leaf_nodes': 48, 'min_impurity_decrease': 0.14369353859644396}. Best is trial 0 with value: -2.2487829376416357e-07.
[I 2024-05-15 10:05:47,578] Trial 1 finished with value: -2.2822535949984513e-07 and parameters: {'n_estimators': 222, 'criterion': 'poisson', 'max_depth': 45, 'min_samples_leaf': 63, 'max_features': None, 'min_weight_fraction_leaf': 1.3948361807311046e-05, 'max_leaf_nodes': 85, 'min_impurity_decrease': 0.011419356166151778}. Best is trial 0 with value: -2.2487829376416357e-07.
[I 2024-05-15 10:06:04,219] Trial 2 finished with value: -2.3544472704806196e-07 and parameters: {'n_estimators': 185, 'crit

In [46]:
optuna.visualization.plot_parallel_coordinate(dtree_study)

In [47]:
# 0.8681371832046793
dtree_study.best_params

{'n_estimators': 164,
 'criterion': 'friedman_mse',
 'max_depth': 57,
 'min_samples_leaf': 1,
 'max_features': None,
 'min_weight_fraction_leaf': 9.641937871123148e-05,
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.00011952699577113836}

In [ ]:
rf_bestParams = {'n_estimators': 164,
 'criterion': 'friedman_mse',
 'max_depth': 57,
 'min_samples_leaf': 1,
 'max_features': None,
 'min_weight_fraction_leaf': 9.641937871123148e-05,
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.00011952699577113836}
rf_model = RandomForestRegressor(**rf_bestParams, random_state=22)
evaluate_model(rf_model)

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 140, 300),
        'criterion': trial.suggest_categorical('criterion', ['friedman_mse', 'squared_error', 'poisson']),
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', None]),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 1e-8, 0.1, log=True),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 25, 100),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 1e-9, 10.0, log=True),
    
        
#             'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
#             'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
#             'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
#             'subsample': trial.suggest_float('subsample', 0.25, 1.0),
            }
    model =  RandomForestRegressor(**params, random_state=22)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    r2 = model.score(X_val, y_val)
    return r2
    
dtree_study = optuna.create_study(direction='maximize')
dtree_study.optimize(objective, n_trials=20)

In [48]:
gb_model = HistGradientBoostingRegressor(max_iter=450,learning_rate=0.09285824534234251,max_leaf_nodes=16,
max_depth=16,min_samples_leaf=41,l2_regularization=0.03528319040165485,max_bins=229,loss='squared_error',random_state=22)
evaluate_model(gb_model)

R2 on test set: 0.8681301820114611 |  Training Time: 8.35 seconds


In [ ]:
et_model = ExtraTreesRegressor(n_estimators=100, random_state=22)
evaluate_model(et_model)

### Tree models don't perform really well.

In [16]:
best_params = {'n_estimators': 1241,
 'learning_rate': 0.020048396891609154,
 'min_split_loss': 0.001187921165440441,
 'max_depth': 87,
 'min_child_weight': 198.63366347882518,
 'max_delta_step': 45.08643901160553,
 'reg_lambda': 0.027961681254236342}
xgb_model = xgb.XGBRegressor(**best_params, random_state=22)
evaluate_model(xgb_model)

R2 on test set: 0.8135588858850391 |  Training Time: 670.68 seconds


In [15]:
lgb_model = lgb.LGBMRegressor(num_leaves=289, learning_rate=0.018820636223047835,n_estimators=283,
                              subsample_for_bin=167805,min_child_samples=62,reg_alpha=8.202839878754152e-07,
                              reg_lambda=0.0010763411222145284,colsample_bytree=0.9994830110430992,
                              subsample=0.8591008767501581,max_depth=9, verbose=-1, random_state=22)
evaluate_model(lgb_model)

R2 on test set: 0.8688950955666532 |  Training Time: 123.48 seconds


In [7]:
cb_model = catboost.CatBoostRegressor(verbose=False, random_state=22)
evaluate_model(cb_model)

R2 on test set: 0.8457158292741519 |  Training Time: 87.97 seconds


### Catboost performs best without tuning

## 2.3. Advanced models with feature engineering

#### In the discussion, it is suggested to have a new feature by summing up all other features.

In [ ]:
# for df in [X_train, X_val, df_test]:
#     df['total_sum'] = df.sum(axis=1)
#     df['special1'] = df['total_sum'].isin(np.arange(72, 76))

In [ ]:
# new_cbmodel = catboost.CatBoostRegressor(verbose=False, random_state=22)
# evaluate_model(new_cbmodel)

#### Another from the discussion, sorting along the feature axis as "feature engineering"

In [ ]:
# def objective(trial):
#     params = {
#         "iterations": 1000,
#         "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
#         "depth": trial.suggest_int("depth", 1, 10),
#         "subsample": trial.suggest_float("subsample", 0.05, 1.0),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#     }

#     model = catboost.CatBoostRegressor(**params, silent=True)
#     model.fit(X_train, y_train)
#     predictions = model.predict(X_val)
#     r2 = model.score(X_val, y_val)
#     return r2

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=30)

In [5]:
# print('Best hyperparameters:', study.best_params)
# print('Best R2:', study.best_value)
best_params = {'learning_rate': 0.05251205350771027, 'depth': 9, 'subsample': 0.7780615991806505, 'colsample_bylevel': 0.9877316058328819, 'min_data_in_leaf': 29}

In [6]:
new_cbmodel = catboost.CatBoostRegressor(**best_params, verbose=False, random_state=22)
evaluate_model(new_cbmodel)

R2 on test set: 0.8688986609942152 |  Training Time: 122.55 seconds


## 3. Submission

In [ ]:
sub = pd.Series(new_cbmodel.predict(df_test), index=df_test.index, name='FloodProbability')
filename = 'submission.csv'
sub.to_csv(filename)
os.system(f"head {filename}")